## 1.Environment Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from surprise import KNNBasic
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from sklearn.metrics import r2_score, precision_score, recall_score, f1_score
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Define path
data_path = '../datasets/student_grade.csv'


In [ ]:
df = pd.read_csv(data_path, low_memory=False)
df

## 2.Data Loading & Preprocessing

In [ ]:
# === 2.1 Load Data ===
df = pd.read_csv(data_path, low_memory=False)

# === 2.2 Transform Data (Wide to Long) ===
id_vars = ['student_id']
df_long = pd.melt(df, id_vars=id_vars, var_name='course', value_name='grade')

# === 2.3 Clean Data ===
# Convert grade to numeric and remove invalid/empty grades
df_long['grade'] = pd.to_numeric(df_long['grade'], errors='coerce')
df_long_cleaned = df_long[(df_long['grade'] > 0.0) & (df_long['grade'].notna())].copy()

# === 2.4 Filter for 'INT' Courses Only ===
# This ensures the model only learns from INT courses
df_long_filtered = df_long_cleaned[df_long_cleaned['course'].astype(str).str.startswith('INT')].copy()

print(f"--- Data Preparation Complete ---")
print(f"Total records after cleaning: {len(df_long_cleaned)}")
print(f"Filtered to INT courses only: {len(df_long_filtered)}")
display(df_long_filtered)


## 3.Split Data to train and test set

In [ ]:
# === 3.1 Load Data into Surprise Dataset ===
# Define rating scale (assuming grades are 1.0 to 4.0)
reader = Reader(rating_scale=(1, 4))
data = Dataset.load_from_df(df_long_filtered[['student_id', 'course', 'grade']], reader)

# === 3.2 Split Data ===
trainset, testset = train_test_split(data, test_size=0.30, random_state=42)

## 4.Model Training (KNNBasic)


In [ ]:
# === 3.3 Train the Model ===
print("--- Training KNN Model (User - User) ---")
sim_user = {
    "name": "cosine",
    "user_based": True
}

algo_user = KNNBasic(sim_options=sim_user)
algo_user.fit(trainset)
print("Training KNN Model (User - User) complete.")
print("-----------------------------------------------------")
print("--- Training KNN Model (Item - Item) ---")
sim_item = {
    "name": "cosine",
    "user_based": False
}

algo_item = KNNBasic(sim_options=sim_item)
algo_item.fit(trainset)
print("Training KNN Model (Item - Item) complete.")



In [ ]:
def recommend_for_user(algo, user_id, n=5):
    items = trainset.all_items()
    items_raw = [trainset.to_raw_iid(i) for i in items]

    predictions = [
        (iid, algo.predict(user_id, iid).est)
        for iid in items_raw
    ]

    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions[:n]

In [ ]:
def similar_items(algo, item_raw_id, k=5):
    inner = algo.trainset.to_inner_iid(item_raw_id)
    neighbors = algo.get_neighbors(inner, k=k)
    raw_ids = [algo.trainset.to_raw_iid(i) for i in neighbors]
    return raw_ids

In [ ]:
recommend_for_user(algo_user, "A246", n=10) # แนะนำวิชาให้ student_id = A246

In [ ]:
similar_items(algo_item, "INT102 WEB TECHNOLOGY") # วิชาที่คล้ายกัน

## 5.Test and evaluate model

In [ ]:
from surprise import accuracy

# Predict on testset
pred_user = algo_user.test(testset)
pred_item = algo_item.test(testset)

# Evaluate
print("=== USER–USER KNN PERFORMANCE ===")
accuracy.rmse(pred_user)
accuracy.mae(pred_user)

print("\n=== ITEM–ITEM KNN PERFORMANCE ===")
accuracy.rmse(pred_item)
accuracy.mae(pred_item)

In [ ]:
from surprise.model_selection import cross_validate

cross_validate(algo_user, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_classification(predictions, threshold=2):
    y_true = []
    y_pred = []

    for p in predictions:
        true_r = p.r_ui
        pred_r = p.est

        # Convert ratings → class
        y_true.append(1 if true_r >= threshold else 0)
        y_pred.append(1 if pred_r >= threshold else 0)

    metrics = {
        "R-score": np.corrcoef(y_true, y_pred)[0, 1],  # Correlation
        "Precision-Macro": precision_score(y_true, y_pred, average='macro'),
        "Recall-Macro": recall_score(y_true, y_pred, average='macro'),
        "F1-Macro": f1_score(y_true, y_pred, average='macro'),
        "Precision-Weighted": precision_score(y_true, y_pred, average='weighted'),
        "Recall-Weighted": recall_score(y_true, y_pred, average='weighted'),
        "F1-Weighted": f1_score(y_true, y_pred, average='weighted')
    }

    return metrics

In [ ]:
metrics_user = evaluate_classification(pred_user, threshold=3)
metrics_user

metrics_item = evaluate_classification(pred_item, threshold=3)
metrics_item

In [ ]:
from collections import defaultdict

# === 4.1 Predict for Unknown Items ===
print("--- Generating Predictions for all missing pairs ---")
anti_testset = trainset.build_anti_testset()
all_predictions = algo_user.test(anti_testset)

# === 4.2 Helper Function for Top-N ===
def get_top_n(predictions, n=5):
    """Return the top-N recommendation for each user from a set of predictions."""
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# === 4.3 Generate Top 5 Recommendations ===
top_n_recommendations = get_top_n(all_predictions, n=5)
print(f"Generated recommendations for {len(top_n_recommendations)} students.")

In [ ]:
# =========================================================
# === Calculate Precision@K and Recall@K ===================
# =========================================================

from collections import defaultdict

# Build Ground Truth:
actual_courses_all = defaultdict(list)
actual_courses_B = defaultdict(list)

for uid, iid, true_r, est, _ in pred_user:
    actual_courses_all[uid].append(iid)

    if true_r >= 3.0:  # grade >= B
        actual_courses_B[uid].append(iid)


def precision_recall_at_k(top_n, actual_dict, K=5):
    precisions = []
    recalls = []

    for uid, user_recs in top_n.items():
        recommended_items = [iid for iid, _ in user_recs[:K]]
        actual_items = actual_dict.get(uid, [])

        if len(actual_items) == 0:
            continue

        true_positives = len(set(recommended_items) & set(actual_items))

        precision = true_positives / K
        recall = true_positives / len(actual_items)

        precisions.append(precision)
        recalls.append(recall)

    precision_avg = sum(precisions) / len(precisions)
    recall_avg = sum(recalls) / len(recalls)
    return precision_avg, recall_avg


# --------- Calculate results for K = 5 and 10 -----------
K_values = [5, 10]

print("\n====================== Precision & Recall ======================")

for K in K_values:
    precision_all, recall_all = precision_recall_at_k(top_n_recommendations, actual_courses_all, K)
    precision_B, recall_B = precision_recall_at_k(top_n_recommendations, actual_courses_B, K)

    print(f"\n========== K = {K} ==========")
    print("-- Using ALL enrolled courses --")
    print(f"Precision@{K}: {precision_all:.4f}")
    print(f"Recall@{K}: {recall_all:.4f}")

    print("\n-- Using ONLY courses with grade >= B --")
    print(f"Precision@{K}: {precision_B:.4f}")
    print(f"Recall@{K}: {recall_B:.4f}")

print("===================================================================")


In [ ]:
from collections import defaultdict, Counter
import matplotlib.pyplot as plt

# === 5.1 Visualization Function ===
def visualize_topk_by_rank(top_n_recommendations, K=5, top_m=10):
    """
    Visualizes the frequency of recommended courses by rank.
    K: Number of recommendations per student.
    top_m: Number of top courses to show in the chart.
    """
    rows = []
    for student_id, recs in top_n_recommendations.items():
        for r, (course, score) in enumerate(recs[:K], start=1):
            rows.append({"rank": r, "course": course})

    if not rows:
        print("No recommendations to visualize.")
        return

    df_viz = pd.DataFrame(rows)
    rank_counters = {r: Counter(df_viz[df_viz["rank"] == r]["course"]) for r in range(1, K+1)}

    for r in range(1, K+1):
        counter = rank_counters[r]
        if not counter: continue

        most_common = counter.most_common(top_m)
        courses = [c for c, _ in most_common]
        counts  = [cnt for _, cnt in most_common]

        plt.figure(figsize=(10, 4))
        plt.bar(courses, counts, color='skyblue')
        plt.title(f"Top {top_m} Courses @ Rank {r}")
        plt.xlabel("Course")
        plt.ylabel("Frequency")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()

# === 5.2 Run Visualization ===
print("--- Visualization of Recommendations ---")
visualize_topk_by_rank(top_n_recommendations, K=5, top_m=10)

# === 5.3 Create Final DataFrame ===
rows = []
for student_id, recs in top_n_recommendations.items():
    for rank, (course, predicted_grade) in enumerate(recs, start=1):
        # Note: Data was filtered for INT in step 2, so this check is just a safeguard
        if isinstance(course, str) and course.startswith("INT"):
            rows.append({
                "student_id": student_id,
                "rank": rank,
                "course": course,
                "predicted_grade": predicted_grade
            })

df_recommendations = pd.DataFrame(rows)

print("\n--- Final Recommendations Preview ---")
display(df_recommendations.head(15))

## 6.Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# 1. Define Mapping: Score -> Letter
# ต้องเรียงลำดับจากน้อยไปมาก เพื่อให้ Matrix สวยงาม
score_to_letter = {
    0.0: 'F', 1.0: 'D', 1.5: 'D+', 2.0: 'C', 2.5: 'C+',
    3.0: 'B', 3.5: 'B+', 4.0: 'A'
}

# สร้าง list ของเกรดที่เป็นไปได้ (เอาไว้ระบุ Labels)
# กรองเอาเฉพาะที่มีใน map (เผื่อกรณีข้อมูลไม่มี F)
valid_scores = sorted(score_to_letter.keys())
valid_labels = [score_to_letter[s] for s in valid_scores]

# 2. Helper function: หาเกรดที่ใกล้ที่สุด
def get_nearest_grade_key(pred_score):
    # หา key (คะแนน) ที่ใกล้ที่สุด
    return min(valid_scores, key=lambda x: abs(x - pred_score))

# 3. Prepare Data
y_true_letters = []
y_pred_letters = []

for pred in pred_user:
    # 3.1 แปลง Actual Grade (r_ui) เป็น Letter
    # ใช้ get_nearest_grade_key เผื่อค่า r_ui มีทศนิยมเพี้ยนเล็กน้อย
    true_score_key = get_nearest_grade_key(pred.r_ui)
    y_true_letters.append(score_to_letter[true_score_key])

    # 3.2 แปลง Predicted Grade (est) เป็น Letter
    pred_score_key = get_nearest_grade_key(pred.est)
    y_pred_letters.append(score_to_letter[pred_score_key])

# 4. Generate Confusion Matrix
# สำคัญ: ต้องใส่ labels=valid_labels เพื่อบังคับลำดับ (D -> A)
cm = confusion_matrix(y_true_letters, y_pred_letters, labels=valid_labels)

# 5. Plot Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=valid_labels,
            yticklabels=valid_labels)

plt.title('Confusion Matrix: Actual vs Predicted Grades')
plt.xlabel('Predicted Grade')
plt.ylabel('Actual Grade')
plt.show()

# 6. Classification Report
print("\n--- Detailed Classification Report ---")
print(classification_report(y_true_letters, y_pred_letters, target_names=valid_labels, zero_division=0,labels=valid_labels))

In [ ]:
from sklearn.metrics import r2_score

y_true_raw = [pred.r_ui for pred in pred_user]
y_pred_raw = [pred.est for pred in pred_user]

# 2. คำนวณ R2 Score
r2 = r2_score(y_true_raw, y_pred_raw)

print(f"R2 Score: {r2:.4f}")

In [ ]:
from surprise.model_selection import GridSearchCV

# data = Dataset.load_from_df(df_long_filtered[['student_id', 'course', 'grade']], reader)

param_grid = {
    'k': [10, 20, 40],
    'sim_options': {
        'name': ['cosine', 'pearson', 'msd', 'euclidean'],
        'user_based': [True, False]
    }
}

gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

In [ ]:
from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import GridSearchCV

# Parameter ที่ใช้ได้สำหรับ KNNBasic
param_grid = {
    'k': [20, 40, 60],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [True, False]   # True=User–User, False=Item–Item
    }
}

gs = GridSearchCV(
    KNNBasic,
    param_grid,
    measures=['rmse', 'mae'],
    cv=5,
    joblib_verbose=3
)

print("🚀 Start GridSearchCV (KNN)...")
gs.fit(data)
print("✅ GridSearchCV Finished")

print("\n🎯 Best RMSE:", gs.best_score['rmse'])
print("🏆 Best parameters:", gs.best_params['rmse'])


In [ ]:
param_grid = {
    'n_factors': [110, 120, 130],
    'n_epochs':  [80, 90, 100],
    'reg_pu':    [0.04, 0.06, 0.08], # Regularization สำหรับ user latent factors
    'reg_qi':    [0.04, 0.06, 0.08], # Regularization สำหรับ item latent factors
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson', 'msd'],
        'user_based': [True, False]   # True=User–User, False=Item–Item
    }
}

gs = GridSearchCV(
    KNNBasic,
    param_grid,
    measures=['rmse', 'mae'],
    cv=5,    # fold cross-validation
    joblib_verbose=3
)

print("🚀 Start GridSearchCV (KNN)...")
gs.fit(data)
print("✅ GridSearchCV Finished")

In [ ]:
import pandas as pd

# แปลงผลลัพธ์ทั้งหมดจาก Grid Search เป็น DataFrame
results_df = pd.DataFrame(gs.cv_results)

# แตก params dict ให้เป็นคอลัมน์
params_df = results_df['params'].apply(pd.Series)

# รวมเข้ากับ metric ที่สนใจ
df = pd.concat([
    params_df,
    results_df[['mean_test_rmse', 'mean_test_mae']]
], axis=1)

# แยกค่า name และ user_based ออกจาก sim_options
df['sim_name'] = df['sim_options'].apply(lambda x: x['name'])
df['user_based'] = df['sim_options'].apply(lambda x: x['user_based'])

print("Best RMSE score:", gs.best_score['rmse'])
print("Best params for RMSE:")
print(gs.best_params['rmse'])

print("\nBest MAE score:", gs.best_score['mae'])
print("Best params for MAE:")
print(gs.best_params['mae'])

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))

sns.lineplot(
    data=df,
    x='k',
    y='mean_test_rmse',
    hue='sim_name',
    marker='o',
    palette='viridis'
)

plt.title('Effect of k on RMSE for KNN')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('RMSE (Lower is Better)')
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# 1. แปลงผลลัพธ์ทั้งหมดจาก Grid Search มาเป็น DataFrame เพื่อให้พลอตกราฟง่าย
results_df = pd.DataFrame.from_dict(gs.cv_results)

# -------------------------------------------------------
# Graph 1: Effect of Epochs (Learning Curve)
# ดูว่า "จำนวนรอบการเรียนรู้" ส่งผลต่อ Error อย่างไร
# -------------------------------------------------------
plt.figure(figsize=(10, 6))

# แกน X: จำนวน Epochs
# แกน Y: ค่า Error (RMSE)
# Hue (สีเส้น): แยกตามจำนวน Factors (ความซับซ้อนของโมเดล)
sns.lineplot(data=results_df, x='param_n_epochs', y='mean_test_rmse',
             hue='param_n_factors', marker='o', palette='viridis')

plt.title('Effect of Epochs on RMSE (Learning Curve)')
plt.xlabel('Number of Epochs')
plt.ylabel('RMSE (Lower is Better)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

df.plot(x='k', y='mean_test_rmse')
plt.xlabel("k")
plt.ylabel("RMSE")
plt.title("K vs RMSE")
plt.show()


In [ ]:
# -------------------------------------------------------
# Graph 2: Heatmap (reg_pu vs reg_qi)
# ดูจุดที่ RMSE ต่ำที่สุดจากค่าคู่ของ regularization
# -------------------------------------------------------
pivot_table = results_df.pivot_table(
    values='mean_test_rmse',
    index='param_reg_pu',    # แกนตั้ง: reg_pu
    columns='param_reg_qi'   # แกนนอน: reg_qi
)

plt.figure(figsize=(8, 6))
sns.heatmap(
    pivot_table,
    annot=True,
    fmt='.4f',
    cmap='Blues_r'  # ยิ่งเข้ม = RMSE ยิ่งต่ำ (ดีกว่า)
)

plt.title('RMSE Heatmap (KNN)')
plt.xlabel('reg_qi (item regularization)')
plt.ylabel('reg_pu (user regularization)')
plt.show()

In [ ]:
# -------------------------------------------------------
# Graph: Heatmap (k vs similarity metric)
# -------------------------------------------------------

# แปลงผล GridSearch ให้อยู่ในรูป DataFrame
results_df = pd.DataFrame.from_dict(gs.cv_results)

# NOTE:
# param_sim_options เป็น dict ดังนั้นต้องแตกค่า metric ออกมาก่อน
results_df['metric'] = results_df['param_sim_options'].apply(lambda x: x['name'])

# ทำ pivot table
pivot_table = results_df.pivot_table(
    values='mean_test_rmse',
    index='param_k',      # แกนตั้ง: k
    columns='metric'      # แกนนอน: similarity metric
)

plt.figure(figsize=(9, 6))
sns.heatmap(
    pivot_table,
    annot=True,
    fmt='.4f',
    cmap='Blues_r'   # ค่าสีเข้มกว่า = ดีกว่า (RMSE ต่ำกว่า)
)

plt.title('RMSE Heatmap (KNN) — k vs Similarity Metric')
plt.xlabel('Similarity Metric')
plt.ylabel('k')
plt.show()


In [ ]:
pivot_table = results_df.pivot_table(
    values='mean_test_rmse',
    index='param_k',
    columns='param_min_k'
)

plt.figure(figsize=(9, 6))
sns.heatmap(
    pivot_table,
    annot=True,
    fmt='.4f',
    cmap='Blues_r'
)

plt.title('RMSE Heatmap (KNN) — k vs min_k')
plt.xlabel('min_k')
plt.ylabel('k')
plt.show()
